## 2. Die gesammelten Daten analysieren


Jetzt haben wir uns viele Daten beschafft, kennen aber noch wenige deren Eigenschaften.

Wir wollen beginnen uns einen Überblick über tatsächlichen Informationen zu beschaffen welche in diesen Daten enthalten sind.

Zuerst wieder die Bibliotheken und Konfigurationen...


In [ ]:
import json
from collections import defaultdict


mode = 1
# 1 = Nationalratswahlen
# 2 = Ständeratswahlen

filename = "nationalrat_raw.json" if mode == 1 else "ständerat_raw.json"

Im folgenden definieren wir eine *Funktion*.
Diese Funktion erlaubt es uns Daten etwas schöner auszugeben

In [ ]:
def print_dict(d: dict):
    maxlen_keys = max(map(len, map(str, d.keys())))
    maxlen_values = max(map(len, map(str, d.values())))
    for key in d.keys():
        print(f"{key:.<{maxlen_keys}}{d[key]:.>{maxlen_values+1}}")

Dann folgt das Laden der Daten.


In [ ]:
candidates = json.load(open(filename, encoding="utf-8"))

Wie viele Personen kandidieren für jede Partei?


In [ ]:
party_count = defaultdict(int)

for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    if party:
        party_count[party] += 1

# Sortieren nach Anzahl Kandidierende pro Partei
party_count = dict(sorted(party_count.items(), key=lambda item: item[1], reverse=True))
print_dict(party_count)

Wie sieht's aus mit dem Frauenanteil der Kandidierenden?


In [5]:
# Ein Dictionary für Frauen und Männer, eins nur für Frauen. Kandidierende ohne Eintrag beim Feld "gender" werden ignoriert
f_and_m = defaultdict(int) # Objekt für Frauen und Männer
f_only = defaultdict(int) # Objekt nur für Frauen

# Iteriert über alle Kandidierenden und zählt Frauen und Männer pro Partei
for candidate in candidates:
    party = candidate.get("partyAbbreviation")
    gender = candidate.get("gender")
    # Wenn die Kandidierende Person einer Partei zugeordnet ist
    if party:
        # Wenn die Kandidierende Person weiblich ist
        if gender == "f":
            f_only[party] += 1
            f_and_m[party] += 1
        # Wenn die Kandidierende Person männlich ist
        if gender == "m":
            f_and_m[party] += 1

# Berechnet den Frauenanteil
female_share = {}
for party_name, women in f_only.items():
    ratio = women / f_and_m[party_name]
    female_share[party_name] = f"{ratio:.0%}"
female_share = dict(sorted(female_share.items(), key=lambda item: item[1], reverse=True))

print_dict(female_share)

NameError: name 'candidates' is not defined